This won't work out of the box, because I can't put GBs of CMIP6 data on GitHub.

In [ ]:
import warnings

import iris
import iris.coord_categorisation
import iris.analysis.cartography
import iris.analysis
import iris.util
import matplotlib.pyplot as pl
import numpy as np
import pandas as pd

In [ ]:
warnings.simplefilter("ignore")

In [ ]:
models = [
    'ACCESS-ESM1-5',
    'BCC-CSM2-MR',
    'CanESM5-CanOE',
    'CanESM5',
    'CESM2',
    'CNRM-ESM2-1',
    'EC-Earth3-CC',
    'GFDL-ESM4',
    'MIROC-ES2L',
    'MPI-ESM1-2-LR',
    'MRI-ESM2-0',
    'NorESM2-LM',
    'UKESM1-0-LL'
]

In [ ]:
table = {model: 'Amon' for model in models}
table['MIROC-ES2L'] = 'AERmon'
table['UKESM1-0-LL'] = 'AERmon'

In [ ]:
pl.rcParams['figure.figsize'] = (9/2.54, 8/2.54)
pl.rcParams['font.size'] = 9
pl.rcParams['font.family'] = 'Arial'
pl.rcParams['xtick.direction'] = 'in'
pl.rcParams['ytick.direction'] = 'in'
pl.rcParams['xtick.minor.visible'] = True
pl.rcParams['ytick.minor.visible'] = True
pl.rcParams['ytick.right'] = True
pl.rcParams['ytick.left'] = True
pl.rcParams['xtick.top'] = True
pl.rcParams['axes.spines.right'] = True
pl.rcParams['axes.spines.left'] = True
pl.rcParams['ytick.labelleft'] = True

In [ ]:
# model = 'ACCESS-ESM1-5'

In [ ]:
# cubes = iris.load(
#         f"/nfs/b0110/Data/cmip6/{model}/esm-hist/*/{table[model]}/co2/*/"
#         f"co2_{table[model]}_{model}_esm-hist_*.nc", "co2"
#     )

In [ ]:
# iris.util.equalise_attributes(cubes)

In [ ]:
# cubes.concatenate_cube()

In [ ]:
# cube

In [ ]:
results = {}

for model in models:
    print(model)
    cubes = iris.load(
        f"/nfs/b0110/Data/cmip6/{model}/esm-hist/*/{table[model]}/co2/*/"
        f"co2_{table[model]}_{model}_esm-hist_*.nc", "co2"
    )
    iris.util.equalise_attributes(cubes)
    iris.util.unify_time_units(cubes)
    cube = cubes.concatenate_cube()
    
    for coord in ['latitude', 'longitude']:
        if not cube.coord(coord).has_bounds():
            cube.coord(coord).guess_bounds()
            
    grid_areas = iris.analysis.cartography.area_weights(cube)
    iris.coord_categorisation.add_year(cube, 'time', name='year')
    annual_means = cube.collapsed(
        ['longitude','latitude'], 
        iris.analysis.MEAN, weights=grid_areas
    ).aggregated_by(['year'], iris.analysis.MEAN)
    results[model] = annual_means[:,0].data

In [ ]:
results

In [ ]:
for model in models:
    if model.startswith('Can'):
        scale = 1
    else:
        scale = 1e6
    results[model] = results[model] * scale

In [ ]:
dataout = pd.DataFrame(results, index=np.arange(1850.5, 2015))
dataout.to_csv('../data/cmip6_esm-hist_co2s.csv')

In [ ]:
ghg_cmip6 = pd.read_csv('../data/meinshausen-et-al-2017/Supplementary_Table_UoM_GHGConcentrations-1-1-0_annualmeans_v2.csv', skiprows=21, index_col=0)
ghg_cmip6['CO2']

In [ ]:
maxco2 = {}
for model in models:
    maxco2[model] = results[model].max()

In [ ]:
maxco2

In [ ]:
models_sorted = {k: v for k, v in sorted(maxco2.items(), key=lambda item: item[1], reverse=True)}
models_sorted

In [ ]:
palette = [
    '#6929c4',
    '#1192e8',
    '#005d5d',
    '#9f1853',
    '#fa4d56',
    '#570408',
    '#198038',
    '#002d9c',
    '#ee538b',
    '#b28600',
    '#009d9a',
    '#012749',
    '#8a3800',
]

colors=dict(zip(models_sorted.keys(), palette))

for i, model in enumerate(models_sorted):
    pl.plot(np.arange(1850.5, 2015), results[model], color=colors[model])
    pl.text(1855, 428-i*10, model, color=colors[model])

pl.text(1917, 418, 'CMIP6 observations\n(Meinshausen et al. 2017)',  fontweight='bold')
pl.plot(np.arange(1850.5, 2015), ghg_cmip6['CO2'], color='k', label='CMIP6 (Meinshausen et al. 2017)', lw=2)
pl.ylabel('ppm')
pl.title('CMIP6 hist-esm CO$_2$ concentrations')

#pl.legend()
pl.xlim(1850, 2015)
pl.ylim(280, 440)
pl.tight_layout()
pl.savefig('../plots/fig4.png', dpi=300)